We want to derive the successive arrays of derivatives of H from it's symbolic expression and have access to a function used to evaluate these arrays. 

In [1]:
import sympy as sp
import numpy as np

In [2]:
q, phi, C0, phi0 = sp.symbols('q, phi, C_0, phi_0', real=True)
states = [q,phi]
def hamiltonian(q,phi):
    return q**2 / (2*C0) + phi**2 / (2*phi0)

expr = hamiltonian(q,phi)
expr = expr.subs(C0, 1)
expr = expr.subs(phi0, 1)


In [3]:
expr

phi**2/2 + q**2/2

Once the only symbols in the expression are the ones for which we need to derive, we can compute derivatives and lambdify the expressions to be able to evaluate the derivatives arrays up to order N.

In [4]:
diff_max_order = 2

diffs = [0]*diff_max_order

diffs[0] = sp.tensor.array.derive_by_array(expr, states)

for i in range(1,diff_max_order):
 
    diffs[i] = sp.tensor.array.derive_by_array(diffs[i-1], states)

Once the derivatives arrays are stored as sympy expression in a list, we can create a list of lambda functions from the expressions using lambdify. The documentation of the generated function can be accessed using .doc.

In [5]:
f_diffs = [sp.lambdify(states,diffs[i]) for i in range(diff_max_order)]
print(f_diffs[0].__doc__)

Created with lambdify. Signature:

func(q, phi)

Expression:

[q, phi]

Source code:

def _lambdifygenerated(q, phi):
    return numpy.array((q, phi))


Imported modules:




Note : for higher odrder derivatives, the lambda expression returns a python list and not a numpy array. But we just need to transform it into a numpy array after evaluation if needed. 

In [7]:
def build_f(order):
    return lambda x : np.array(f_diffs[order](*x))
f_diffs_np = [build_f(i)  for i in range(diff_max_order)]

In [10]:
f_diffs_np[2]([[0,1],[1,2]])

IndexError: list index out of range